In [ ]:
!pip install tensorflow==1.15
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==1.15
!pip install transformers
!pip install numpy==1.19.5
!pip install 'h5py==2.10.0'

In [ ]:
import torch
import transformers as ppb
import tensorflow as tf
import tensorflow_hub as hub
import natsort

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Activation, Embedding, Flatten, MaxPooling1D, Dropout, Conv1D, Input, LSTM, SpatialDropout1D, Bidirectional

In [ ]:
import numpy
print(numpy.__version__)

In [ ]:
aladag = False
reddit_500 = True

In [ ]:
import h5py
print(h5py.__version__)

#IF the numpy version not equal to 1.19.5 and h5py version not equal to 2.10.0 restart the runtime and start again from cell 2

In [ ]:
from google.colab import auth
auth.authenticate_user()
!curl https://sdk.cloud.google.com | bash

!gcloud init

In [ ]:
!mkdir training_directory

In [ ]:
#Choose the path to the dataset and embeddings
!gsutil -m cp gs://masterthesisbert/embeddings_right/PsychBERT_con_long/MyBERT_wide_full-training-features_2D_labeled_Alada* /content/training_directory/

In [ ]:
import os
import natsort
arr = os.listdir("/content/training_directory")
arr = natsort.natsorted(arr)
arr

In [ ]:
# load numpy array from csv file
from numpy import loadtxt
import numpy as np

third_dimension = 768

arrays = []

if aladag:

    length = 10

elif reddit_500:

    length = 5

for i in range(len(arr[0:length])):
    loaded_arr = np.loadtxt('/content/training_directory'+"/"+arr[i])
    arrays.append(loaded_arr)
    print(i)

In [ ]:
concatenated_array = np.array([]).reshape(0, 768)
# Create an array to return to

for array in arrays:

    concatenated_array = np.concatenate([concatenated_array, array], axis=0)
data_train_x = concatenated_array

In [ ]:
import pandas as pd
import tensorflow as tf

if aladag:  

    !gsutil cp gs://masterthesisbert/Aladag_sample_preprocessed.csv  /content/Aladag_sample_preprocessed.csv
    
    df_name = "/content/Aladag_sample_preprocessed.csv"
    df = pd.read_csv(df_name)

    !gsutil cp gs://masterthesisbert/Aladag_labeled_preprocessed.csv  /content/Aladag_labeled_preprocessed.csv

    df_name2 = "/content/Aladag_labeled_preprocessed.csv"
    df2 = pd.read_csv(df_name2)
    df2

    train_y_tensor = torch.tensor(df[0:10000].binary_annotation.values)
    train_y_tensor2 = torch.tensor(df2[0:10000].binary_annotation.values)

elif reddit_500:
  
    !gsutil cp gs://masterthesisbert/reddit_500_final_val.csv  /content/reddit_500_final_val.csv

    df_name2 = "/content/reddit_500_final_val.csv"
    df2 = pd.read_csv(df_name2)
    df2

    !gsutil cp gs://masterthesisbert/reddit_500_final_train.csv  /content/reddit_500_final_train.csv

    df_name = "/content/reddit_500_final_train.csv"
    df = pd.read_csv(df_name)
    df

    train_y_tensor = torch.tensor(df[0:10000].Label.values)
    train_y_tensor2 = torch.tensor(df2[0:10000].Label.values)

In [ ]:
permutation_done = True
if permutation_done and reddit_500:
    !gsutil cp gs://masterthesisbert/parameter_optimization/permutation.csv  /content/permutation.csv
    from numpy import loadtxt
    p = loadtxt('permutation.csv', delimiter=',')
    concatenated_array, train_y_tensor_permut = data_train_x[p.astype(int)], train_y_tensor[p.astype(int)]
    data_train_x, train_y_tensor_permut = data_train_x[p.astype(int)], train_y_tensor[p.astype(int)]

In [ ]:
import pandas as pd
import numpy as np

from tensorflow import keras
from tensorflow.python.keras.layers import Input, Dense,RepeatVector, TimeDistributed, Dense, Dropout, LSTM, Bidirectional
from tensorflow.python.keras.models import Sequential

import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Dimensionality reduction

In [ ]:
def summarize_results(grid_result, model_name, result_to_file = True):
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

    if result_to_file == True:
        string_to_save = f"Best: {grid_result.best_score_} using {grid_result.best_params_} \n" 
        for mean, stdev, param in zip(means, stds, params):
            string_to_save = string_to_save + f"{mean} ({stdev}) with: {param} \n" 
        text_results = open(model_name + ".txt", "w")
        n = text_results.write(string_to_save)
        text_results.close()

from tensorflow.keras.models import model_from_json

In [ ]:
# Conctruct autoencoder for training test to find anomalous days in the data
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV
import os
import random as rn 
from tensorflow.compat.v1.keras import backend as K

from sklearn.metrics import mean_squared_error


from google.colab import files
number = 2
os.environ['PYTHONHASHSEED']=str(number)
#Set random seed for numpy, python and tensorflow
np.random.seed(number)
rn.seed(number)
tf.set_random_seed(number)
# Set the number of threads to 1 
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)


#Novelty detection approach.


In [ ]:
def create_dense_autoencoder(data_train_x = data_train_x):

    # Fully Dense Network
    model_autoencoder_dense = Sequential()

    model_autoencoder_dense_path = "model_autoencoder_dense"

    #model_autoencoder_dense.add(Input(shape=(data_train_x.shape[1], data_train_x.shape[2],)))

    model_autoencoder_dense.add(Dense(768/2, activation='sigmoid', kernel_initializer=keras.initializers.glorot_uniform(seed=number), input_shape=(data_train_x.shape[1],)))
    model_autoencoder_dense.add(Dense(1, activation='sigmoid', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))
    model_autoencoder_dense.add(Dense(768/2, activation='tanh', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))
    model_autoencoder_dense.add(Dense(768, activation='tanh', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))
    #model_autoencoder_dense.add(keras.layers.TimeDistributed(keras.layers.Dense(data_train_x.shape[2],kernel_initializer=keras.initializers.glorot_uniform(seed=number) )))
        
    optimizer = keras.optimizers.Adam(learning_rate=0.00001)


    model_autoencoder_dense.compile(optimizer=optimizer,
                  loss='mse')

    #mc = ModelCheckpoint(model_autoencoder_dense_path + ".h5", monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

    model_autoencoder_dense.summary()

    return model_autoencoder_dense

model_autoencoder_Dense = KerasRegressor(build_fn=create_dense_autoencoder, epochs=10)
param_grid = dict(epochs=[120])
grid = GridSearchCV(estimator=model_autoencoder_Dense, param_grid=param_grid, cv=5)

In [ ]:
model_autoencoder_dense = create_dense_autoencoder(data_train_x)
#choose the best parameters as from the grid search 
model_autoencoder_dense.fit(data_train_x, data_train_x, epochs=120)

model_autoencoder_dense.save('model_autoencoder_dense.h5')
new_model = tf.keras.models.load_model('model_autoencoder_dense.h5')

In [ ]:
from sklearn.metrics import mean_squared_error
X_pred = model_autoencoder_dense.predict(data_train_x)

#obtain mean square error for each day between predicted 24 hours and actual 24 hours 

train_mse_loss = np.mean(np.square(X_pred - data_train_x), axis=1)

train_mse_loss.shape

In [ ]:
import skimage.io
import skimage.color
import skimage.filters

In [ ]:
# perform automatic thresholding
t = skimage.filters.threshold_otsu(train_mse_loss)
print("Found automatic threshold t = {}.".format(t)) 

Found automatic threshold t = 0.11656135659109973.


In [ ]:
from google.colab import files
#3error_sum = np.sum(train_mse_loss, axis=1)
plt.hist(train_mse_loss, bins=80);
plt.axvline(t, color='red', linestyle='dashed', linewidth=1)
plt.ylabel('NofObservations')
#plt.ylim(y_min, y_max)
plt.xlabel('MSE')

In [ ]:
anomaly_day_indices = [idx for idx, val in enumerate(train_mse_loss) if val > t]
full_train_without_anomalies = np.delete(data_train_x, anomaly_day_indices, axis=0)

# Data is ready for a Classification
full_train_without_anomalies.shape

(4818, 768)

In [ ]:
# Fully Dense Network

model_autoencoder_dense_path2 = "model_autoencoder_dense2"
inputs = tf.keras.Input(shape=(data_train_x.shape[1],))
encoder_1 = Dense(data_train_x.shape[1]/2, activation='sigmoid', kernel_initializer=keras.initializers.glorot_uniform(seed=number))(inputs)

encoder_2 = Dense(1, activation='sigmoid', kernel_initializer=keras.initializers.glorot_uniform(seed=number))(encoder_1)

decoder_1 = Dense(data_train_x.shape[1]/2, activation='tanh', kernel_initializer=keras.initializers.glorot_uniform(seed=number))(encoder_2)

decoder_2 = Dense(data_train_x.shape[1], activation='tanh', kernel_initializer=keras.initializers.glorot_uniform(seed=number))(decoder_1)


model_autoencoder_dense2 = Model(inputs = inputs, outputs = decoder_2)

optimizer = keras.optimizers.Adam(learning_rate=0.00001)

model_autoencoder_dense2.compile(loss="mse", optimizer=optimizer)
                                 

#mc = ModelCheckpoint(model_autoencoder_dense_path + ".h5", monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

model_autoencoder_dense2.summary()

In [ ]:
model_autoencoder_dense2.fit(full_train_without_anomalies, full_train_without_anomalies, epochs=120)

model_autoencoder_dense2.save('model_autoencoder_dense2.h5')
model_autoencoder_dense2= tf.keras.models.load_model('model_autoencoder_dense2.h5')

In [ ]:
from google.colab import files 
from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

In [ ]:
def get_normal_rows(data_train_x, model_autoencoder, inputs, encoder_2, eps=0.005, shape=10000):

    X_pred = model_autoencoder.predict(data_train_x)
    #obtain mean square error for each day between predicted 24 hours and actual 24 hours 
    train_mse_loss_final = np.mean(np.square(X_pred - data_train_x), axis=1)

    # perform automatic thresholding
    t_2 = skimage.filters.threshold_otsu(train_mse_loss_final)
    print("Found automatic threshold t = {}.".format(t_2)) 

    #plot a histogram
    #3error_sum = np.sum(train_mse_loss, axis=1)
    plt.hist(train_mse_loss_final, bins=80);
    plt.axvline(t_2, color='red', linestyle='dashed', linewidth=1)

    plt.ylabel('NofObservations')
    #plt.ylim(y_min, y_max)
    plt.xlabel('MSE')

    encoder = Model(inputs , encoder_2)
    encoded_train = encoder.predict(data_train_x)
    encoded_train_reshaped = encoded_train.reshape(shape, 1)
    plt.scatter(train_mse_loss_final, encoded_train)
    plt.show()
    merged_list = []

    for l in encoded_train:
        
        merged_list.append(l[0])

    merged_list_new = numpy.array(merged_list)
    merged_list_new.shape
    Z = np.dstack((train_mse_loss_final, merged_list_new))
    merged_list_2 = []

    for l in range(len(Z[0])):
        
        merged_list_2.append(Z[0][l])
    #Clustering
    nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(merged_list_2)
    distances, indices = nbrs.kneighbors(merged_list_2)

    sns.set()
    distances = np.sort(distances, axis=0)
    distances = distances[:,1]
    plt.plot(distances)
    #The optimal point for eps will the found at the point of maximal curvature
    
    m = DBSCAN(eps, min_samples=5)
    m.fit(merged_list_2)
    clusters = m.labels_
    # taking an input list
    l1 = []
    
    # taking an counter
    n_clusters = 0
   
    # traversing the array
    for item in clusters:
        if item not in l1:
            n_clusters += 1
            l1.append(item)
    
    # printing the output
    print("No of unique items are:", n_clusters)

    def find(lst, a):
        return [i for i, x in enumerate(lst) if x==a]
    dict_error = {}
    cluster_labels = list(range(-1, n_clusters-1))

    for i in range(-1, n_clusters-1):
        cluster_i_set = []
        for k in find(clusters, i):
            cluster_i_set.append(merged_list_2[k])
        #cluster_i_set = merged_list_2[find(clusters, i)]
        len_cluster_i_set = len(cluster_i_set)
        error_count = 0
        for j in range(len_cluster_i_set):
            if cluster_i_set[j][0]>t_2:
              error_count += 1

        dict_error[i] = error_count
    occurences_of_cluster = {}
    novelty_normal = {}

    def return_all_normal_indices(clusters, key):
        return [i for i, e in enumerate(clusters.tolist()) if e == key]

    for i in range(-1, n_clusters-1):
        number = clusters.tolist().count(i)
        if (number * 0.7 <= dict_error[i] ):
            novelty_normal[i] = "novelty"
        else:
            novelty_normal[i] = "normal"
    normal_clusters = []
    normal_indices = []
    for key, item in novelty_normal.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
        if item == "normal":
            normal_clusters.append(key)
            normal_indices = normal_indices + return_all_normal_indices(clusters, key)

    return normal_indices

In [ ]:
normal_indices = get_normal_rows(data_train_x, model_autoencoder_dense2, inputs, encoder_2, shape = 10000)

In [ ]:
len(normal_indices)

In [ ]:
# open file in write mode. Actually the name of the embeddigs and the dataset would be cool. 
with open(r'BERT_reddit_500_noisy1.txt', 'w') as fp:
    for item in normal_indices:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

In [ ]:
!gsutil cp /content/BERT_reddit_500_noisy1.txt gs://masterthesisbert/anomaly_indices/

Copying file:///content/BERT_reddit_500_noisy1.txt [Content-Type=text/plain]...
/ [1 files][ 18.3 KiB/ 18.3 KiB]                                                
Operation completed over 1 objects/18.3 KiB.                                     
